In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Hyperparameters
num_epochs = 50
batch_size = 128  # Increased batch size
learning_rate = 0.001  # Reduced learning rate

In [4]:
# Data preprocessing
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # CIFAR-10 specific stats
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [5]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:02<00:00, 57.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
class ConvNet(nn.Module):
  def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(256 * 4 * 4, 512)  # Adjusted for new feature map size
        self.fc2 = nn.Linear(512, 10)

  def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 256 * 4 * 4)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x



In [7]:
model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=3, factor=0.5)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if (i+1) % 2000 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


  model.eval()
  val_loss = 0.0
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in test_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      val_loss += criterion(outputs, labels).item()
      _, predicted = outputs.max(1)
      total += labels.size(0)
      correct += predicted.eq(labels).sum().item()

  val_loss /= len(test_loader)
  accuracy = 100.0 * correct / total
  print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")
  scheduler.step(val_loss)

print("Finished Training")

Epoch [1/50], Validation Loss: 1.2808, Accuracy: 52.88%
Epoch [2/50], Validation Loss: 1.1829, Accuracy: 57.75%
Epoch [3/50], Validation Loss: 1.0081, Accuracy: 63.68%
Epoch [4/50], Validation Loss: 1.1752, Accuracy: 59.61%
Epoch [5/50], Validation Loss: 0.8429, Accuracy: 69.55%
Epoch [6/50], Validation Loss: 0.7937, Accuracy: 71.67%
Epoch [7/50], Validation Loss: 0.7748, Accuracy: 72.08%
Epoch [8/50], Validation Loss: 0.7451, Accuracy: 73.46%
Epoch [9/50], Validation Loss: 0.8365, Accuracy: 71.44%
Epoch [10/50], Validation Loss: 0.6890, Accuracy: 75.40%
Epoch [11/50], Validation Loss: 0.6979, Accuracy: 76.09%
Epoch [12/50], Validation Loss: 0.6540, Accuracy: 76.94%
Epoch [13/50], Validation Loss: 0.6328, Accuracy: 78.11%
Epoch [14/50], Validation Loss: 0.6243, Accuracy: 78.37%
Epoch [15/50], Validation Loss: 0.6038, Accuracy: 79.04%
Epoch [16/50], Validation Loss: 0.5766, Accuracy: 80.14%
Epoch [17/50], Validation Loss: 0.6086, Accuracy: 79.05%
Epoch [18/50], Validation Loss: 0.6124, 